# This program is created by ...

In [3]:
# Import all the necessary packages:
import numpy as np                                     # type: ignore
import matplotlib.pyplot as plt                        # type: ignore
from datasets import load_dataset                      # type: ignore
import cv2                                             # type: ignore
from sklearn.ensemble import RandomForestClassifier    # type: ignore
from sklearn.model_selection import train_test_split   # type: ignore
from sklearn.metrics import accuracy_score             # type: ignore


%matplotlib inline

## Download the dataset. This will probably take 4-7 minutes

#### Download straight from HuggingFace, then convert the training data into a pandas DataFrame

In [ ]:
# Download the dataset
ds = load_dataset("marmal88/skin_cancer")

# print(ds.keys()) yields:
#     dict_keys(['train', 'validation', 'test'])

#### Render an image

In [ ]:
# df = ds["train"].to_pandas()
# image_bytes = df.iloc[0]["image"]["bytes"]

# # Convert the byte data into a NumPy array
# nparr = np.frombuffer(image_bytes, np.uint8)

# # Decode the image array into an image using OpenCV
# image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

# # Convert the image from BGR (OpenCV's default) to RGB for correct display
# image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Display the image using matplotlib
# plt.imshow(image_rgb)
# plt.axis('off')  # Hide axis
# plt.show()

## Implement the Sobel and Canny edge detectors that we will use across the training dataset

#### Sobel

In [ ]:
def ApplySobel(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = cv2.magnitude(sobelx, sobely)
    return cv2.normalize(sobel_combined, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

#### Canny

In [ ]:
def ApplyCanny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return cv2.Canny(gray, 100, 200)

## Now, apply Sobel and Canny to each image in the training set

In [ ]:
dfTrain = ds["train"].to_pandas()

sobelImagesTrain = []
cannyImagesTrain = []

for index, row in dfTrain.iterrows():
    image = cv2.cvtColor(cv2.imdecode(np.frombuffer(row["image"]["bytes"], np.uint8), cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
    
    sobelImage = ApplySobel(image)
    cannyImage = ApplyCanny(image)
    
    sobelImagesTrain.append(sobelImage)
    cannyImagesTrain.append(cannyImage)
    break

# Display sample images
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(image)
axes[0].set_title("Original")
axes[1].imshow(sobelImage, cmap='gray')
axes[1].set_title("Sobel Edge Detection")
axes[2].imshow(cannyImage, cmap='gray')
axes[2].set_title("Canny Edge Detection")
for ax in axes:
    ax.axis("off")
plt.show()

#### Train the Sobel ML model on the Sobel train images

In [ ]:
# To do

#### Train the Canny ML model on the Canny train images

In [ ]:
# To do

## Apply the respective ML model to the training set * IMPROVE THIS TITLE * 

#### Apply Sobel and Canny to the test set

In [ ]:
dfTest = ds["test"].to_pandas()

sobelImagesTest = []
cannyImagesTest = []

for index, row in dfTest.iterrows():
    image = cv2.cvtColor(cv2.imdecode(np.frombuffer(row["image"]["bytes"], np.uint8), cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
    
    sobelImage = ApplySobel(image)
    cannyImage = ApplyCanny(image)
    
    sobelImagesTest.append(sobelImage)
    cannyImagesTest.append(cannyImage)

# Now we have a list of images that have Sobel and Canny applied to them. These images are stored in sobelImagesTest and cannyImagesTest

#### Run the Sobel ML model on the Sobel test set to tune parameters

In [ ]:
# To do

#### Run the Canny ML model on the Canny test set to tune parameters

In [ ]:
# To do

## Finally, compare both models on data in the validation set, after all training and testing

#### Apply Sobel and Canny to the validation set

In [ ]:
dfVal = ds["validation"].to_pandas()

sobelImagesVal = []
cannyImagesVal = []

for index, row in dfVal.iterrows():
    image = cv2.cvtColor(cv2.imdecode(np.frombuffer(row["image"]["bytes"], np.uint8), cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
    
    sobelImage = ApplySobel(image)
    cannyImage = ApplyCanny(image)
    
    sobelImagesVal.append(sobelImage)
    cannyImagesVal.append(cannyImage)

#### Run both ML models on their respective images

In [ ]:
# Run Sobel ML model on sobelImagesVal
# Run Canny ML model on cannyImagesVal

#### Get metrics on the performance of both models

In [ ]:
# Get the Sobel model's accuracy, f1 score, etc.
# Get the Canny model's accuracy, f1 score, etc.

# This is the last step. If Sobel is more accurate, better score etc. then we conclude that Sobel is better